# Preparación

Carga de datasets y formulación de funciones a utilizar en el análisis.

### RMSE (Root Mean Squared Error):

Root Mean Squared Error entre dos arrays ignorando valores infinitos y nulos.

### FFT

Estimación simple de la densidad espectral de potencia (PSD)

### Dominant Power Band

Toma un espectro (frecuencias + PSD) y devuelve un intervalo [f_lo, f_hi] que contiene aproximadamente una fracción q de la potencia total.



In [ ]:

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px

from scipy import signal
from sklearn.cluster import DBSCAN
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

import networkx as nx
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.statespace.sarimax import SARIMAX

def rmse(y_true, y_pred) -> float:
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    m = np.isfinite(y_true) & np.isfinite(y_pred)
    return float(np.sqrt(mean_squared_error(y_true[m], y_pred[m])))

def fft_psd(x: np.ndarray, fs: float = 1.0):
    x = np.asarray(x, dtype=float)
    x = x[np.isfinite(x)]
    x = x - np.mean(x) if len(x) else x
    n = len(x)
    freqs = np.fft.rfftfreq(n, d=1.0/fs)
    X = np.fft.rfft(x)
    psd = (np.abs(X) ** 2) / max(n, 1)
    return freqs, psd

def dominant_power_band(freqs: np.ndarray, psd: np.ndarray, q: float = 0.9, fmin_exclude: float = 0.0):
    freqs = np.asarray(freqs)
    psd = np.asarray(psd)
    m = freqs > fmin_exclude
    freqs2, psd2 = freqs[m], psd[m]
    if len(freqs2) < 3 or np.sum(psd2) <= 0:
        return (np.nan, np.nan)
    order = np.argsort(freqs2)
    freqs2, psd2 = freqs2[order], psd2[order]
    csum = np.cumsum(psd2)
    total = csum[-1]
    lo = np.searchsorted(csum, (1-q)*total/2)
    hi = np.searchsorted(csum, (1+q)*total/2)
    lo = int(np.clip(lo, 0, len(freqs2)-1))
    hi = int(np.clip(hi, 0, len(freqs2)-1))
    return (float(freqs2[lo]), float(freqs2[hi]))

# Carga
df_agro_clean = pd.read_csv("agro_clean.csv")
df_agro_noise = pd.read_csv("agro_noise.csv")
df_ener_clean = pd.read_csv("ener_clean.csv")
df_ener_noise = pd.read_csv("ener_noise.csv")

print("AGRO clean:", df_agro_clean.shape, "| cols:", len(df_agro_clean.columns))
print("AGRO noise:", df_agro_noise.shape, "| cols:", len(df_agro_noise.columns))
print("ENER clean:", df_ener_clean.shape, "| cols:", len(df_ener_clean.columns))
print("ENER noise:", df_ener_noise.shape, "| cols:", len(df_ener_noise.columns))


AGRO clean: (2000, 14) | cols: 14
AGRO noise: (2000, 14) | cols: 14
ENER clean: (2000, 14) | cols: 14
ENER noise: (2000, 14) | cols: 14


**Bloque 1:**  
1) Visualizamos sensores en un `scatter_mapbox` usando coordenadas del `agro_clean`.  
2) Codificamos **color = NDVI (Agro_5)** y **tamaño = Humedad (Agro_1)** para leer biomasa vs humedad en el espacio.  
3) Para responder si hay *patrón espacial* de biomasa baja, filtramos el 20% inferior de NDVI (umbral p20) y aplicamos DBSCAN, identificando clusters donde NDVI bajo se agrupa geográficamente.  

**Conclusión**
Aunque, visualmente, aparentemente, se presentan clusters pequeños, la diferencia en NDVI no es significativa en comparación a cualquier zona fuera de estos como para concluir que la baja biomasa tiene un patrón espacial.

In [18]:
latitud = "Latitude"
longitud = "Longitude"
ndvi_col = "Agro_5"
hum_col  = "Agro_1"

df_map = df_agro_clean.reset_index(drop=False).copy()

center = dict(
    lat=float(pd.to_numeric(df_map["Latitude"], errors="coerce").mean()),
    lon=float(pd.to_numeric(df_map["Longitude"], errors="coerce").mean())
)

fig = px.scatter_mapbox(
    df_map,
    lat=latitud, lon=longitud,
    color=ndvi_col,
    size=hum_col,
    size_max=18,
    zoom=9,
    center=center,
    mapbox_style="open-street-map",
    hover_data=[ndvi_col, hum_col]
)
fig.update_layout(margin=dict(l=0,r=0,t=0,b=0))
fig.show()

# Clustering de baja biomasa: NDVI bajo (p20) + DBSCAN geoespacial
thr = float(df_map[ndvi_col].quantile(0.20))
df_low = df_map.loc[df_map[ndvi_col] <= thr].copy()

coords = df_low[[latitud, longitud]].apply(pd.to_numeric, errors="coerce")
coords = coords.dropna()
coords_rad = np.radians(coords.to_numpy())

# eps ~ 1.5 km (ajustable)
earth_km = 6371.0
eps_km = 1.5
eps = eps_km / earth_km

db = DBSCAN(eps=eps, min_samples=10, metric="haversine")
labels = db.fit_predict(coords_rad)

df_low = df_low.loc[coords.index].copy()
df_low["cluster_low_ndvi"] = labels

summary = (
    df_low[df_low["cluster_low_ndvi"] != -1]
    .groupby("cluster_low_ndvi")
    .agg(n_points=("cluster_low_ndvi","size"),
         ndvi_mean=(ndvi_col,"mean"),
         lat_mean=(latitud,"mean"),
         lon_mean=(longitud,"mean"))
    .sort_values(["n_points","ndvi_mean"], ascending=[False, True])
)
print(f"Umbral NDVI bajo (p20): {thr:.4f}")
print("Clusters de NDVI bajo (DBSCAN; -1 = ruido/no cluster):")
print(summary)

fig2 = px.scatter_mapbox(
    df_low,
    lat=latitud, lon=longitud,
    color="cluster_low_ndvi",
    zoom=9,
    center=center,
    mapbox_style="open-street-map",
    hover_data=[ndvi_col, hum_col, "cluster_low_ndvi"]
)
fig2.update_layout(margin=dict(l=0,r=0,t=0,b=0))
fig2.show()

Umbral NDVI bajo (p20): 0.8354
Clusters de NDVI bajo (DBSCAN; -1 = ruido/no cluster):
                  n_points  ndvi_mean  lat_mean   lon_mean
cluster_low_ndvi                                          
1                       15   0.740094  6.219740 -75.320232
3                       14   0.770666  6.287724 -75.363507
2                       13   0.743680  6.176799 -75.345608
4                       13   0.751648  6.288079 -75.312613
8                       11   0.731319  6.128820 -75.411111
9                       10   0.724711  6.143662 -75.382305
7                       10   0.740585  6.246821 -75.340162
0                       10   0.742114  6.176963 -75.461531
5                       10   0.755512  6.188488 -75.415411
6                       10   0.756665  6.142275 -75.460214


**Bloque 2:**  
1) Corremos **ADF** sobre todas las columnas `Ener_*` del `ener_clean` para identificar estacionariedad (H0: raíz unitaria ⇒ no estacionaria).  
2) Para series con p-value > 0.05, calculamos media móvil y varianza móvil con ventana 50 para evidenciar cambios de nivel o cambios de volatilidad.  
3) Para `Ener_5` (Costo del Gas), buscamos drift o random walk con dos señales:  
   - ADF en nivel suele fallar (no estacionaria).  
   - ADF en diferencias suele pasar (estacionaria).  

**Conclusión**
La evidencia visual indica que Ener_5 se comporta como un random walk con drift positivo. La serie no oscila alrededor de una media fija sino que presenta una tendencia ascendente., y la media móvil se desplaza hacia arriba también, lo cual es típico de no estacionariedad. En el gráfico de la primera diferencia, los incrementos son estables alrededor de una linea horizontal, pero esa línea está por encima de cero (mean(Δ)=0.0106), lo que visualmente confirma un drift sobre un proceso tipo random walk. Se consideró la posibilidad de que la media fuese tan pequeña que pudiese ser obviada, pero al comparar con la desviación estandar y teniendo en cuenta la cantidad de pasos, a largo plazo dería significativa.


In [ ]:

ener_cols = [c for c in df_ener_clean.columns if str(c).startswith("Ener_")]
print("Ener_* detectadas:", sorted(ener_cols))

adf_rows = []
for c in sorted(ener_cols):
    s = pd.to_numeric(df_ener_clean[c], errors="coerce").dropna()
    stat, pval, usedlag, nobs, crit, icbest = adfuller(s, autolag="AIC")
    adf_rows.append({"serie": c, "adf_stat": stat, "p_value": pval, "nobs": nobs})

adf_df = pd.DataFrame(adf_rows).sort_values("p_value")
print("\nADF (ener_clean) ordenado por p-value:")
print(adf_df)

non_stationary = adf_df.loc[adf_df["p_value"] > 0.05, "serie"].tolist()
print("\nNo estacionarias (p>0.05):", non_stationary)

# Rolling mean/var (50) para las no estacionarias
W = 50
for c in non_stationary:
    rmean = df_ener_clean[c].rolling(W).mean()
    rvar  = df_ener_clean[c].rolling(W).var()

    plt.figure(figsize=(12,4))
    plt.plot(df_ener_clean[c].values, label=f"{c} (nivel)")
    plt.plot(rmean.values, label=f"Media móvil ({W})")
    plt.title(f"{c}: nivel + media móvil")
    plt.legend(); plt.grid(alpha=0.25); plt.show()

    plt.figure(figsize=(12,4))
    plt.plot(rvar.values, label=f"Varianza móvil ({W})")
    plt.title(f"{c}: varianza móvil")
    plt.legend(); plt.grid(alpha=0.25); plt.show()


s = pd.to_numeric(df_ener_clean["Ener_5"], errors="coerce").dropna()
ds = s.diff().dropna()

p_level = adfuller(s, autolag="AIC")[1]
p_diff  = adfuller(ds, autolag="AIC")[1]
mu_diff = float(ds.mean())

print(f"\nEner_5 ADF p-value (nivel): {p_level:.6g}")
print(f"Ener_5 ADF p-value (Δ):     {p_diff:.6g}")
print(f"Media(ΔEner_5):             {mu_diff:.6g}")

rmean = s.rolling(W).mean()
plt.figure(figsize=(12,4))
plt.plot(s.values, label="Ener_5 (nivel)")
plt.plot(rmean.values, label=f"Media móvil ({W})")
plt.title("Ener_5: evidencia visual de drift (nivel cambia; media móvil se desplaza)")
plt.legend(); plt.grid(alpha=0.25); plt.show()

plt.figure(figsize=(12,4))
plt.plot(ds.values, label="ΔEner_5")
plt.axhline(mu_diff, linestyle="--", linewidth=1, label=f"mean(Δ)={mu_diff:.4f}")
plt.title("Ener_5: primera diferencia (drift si mean(Δ) ≠ 0 y Δ es estacionaria)")
plt.legend(); plt.grid(alpha=0.25); plt.show()


In [ ]:
ds.describe()